## Fixing/Modifying individual file (EIGENSTRAT)

### Fixing populations after convertf

Individual file (commonly *.ind) contains population information for samples.
Here: collect 4 different Metadata files, as pandas DFs, combine them and use as lookup, when fixing the original *.ind file that did not have that info after convertf.

### ind file variations
The 2. purpose is to explore population definitions and accordingly write
Also, writing separate popfiles (as used by qp3Pop) for Northern and Southern Dispersal Route. Finally, creating ind file for use popfile


In [1]:
from collections import Counter

import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
import re
def readZallouaMetadata(manifest, selcols, colnames=['Id','Region']):    
    regionalContext = pd.read_csv(manifest, skiprows=8, sep=',', skip_blank_lines=True)
    regionalContext = regionalContext.iloc[:,selcols]
    regionalContext.columns = colnames
    regionalContext.set_index('Id', inplace=True)
    return regionalContext.dropna(subset=colnames[1:])
yemenMeta = readZallouaMetadata("Metadata/3577stdy_manifest_3450_190315.csv.gz", [2,3], colnames=['Id','RegionId'])
yemenMeta["Region"] = [re.findall('\D+', sup)[0] for sup in yemenMeta['RegionId']]
yemenMeta = yemenMeta[['Region']]
yemenMeta = yemenMeta[yemenMeta.Region!='Empty']
chadContext = readZallouaMetadata("Metadata/yemcha_manifest_3558_160415.csv.gz", [2,8])
pakistan = readZallouaMetadata("Metadata/sc_egyptlc_manifest_1764_050613.csv.gz", [2,8])
pakistan.head()

,Region
Id,
SC_EGYPTLC5544382,Pakistan
SC_EGYPTLC5544383,Pakistan
SC_EGYPTLC5544384,Pakistan
SC_EGYPTLC5544385,Pakistan
SC_EGYPTLC5544386,Pakistan


In [15]:
yemenMeta.head()

,Region
Id,
3577STDY6068360,Dal
3577STDY6068361,Ibb
3577STDY6068362,Rsa
3577STDY6068363,Rsa
3577STDY6068364,Rsa


In [16]:
reichMeta=pd.read_csv("Reich/v44.3_HO_public.anno", sep='\t')
reichMeta.columns = ['Index', 'Id', 'Id2',
       'Publication', 'contact',
       'Date',
       'Full_Date',
       'Group_Label', 'Locality', 'Region', 'Lat', 'Long',
       'Data source', 'Cov_autosm',
       'SNPs_autosm', 'Sex',
       'Library_type',
       'ASSESSMENT']

reichMetaNow = reichMeta[reichMeta.Date==0]
reichMetaAncient = reichMeta[reichMeta.Date>1000]

for df in [reichMetaAncient, reichMetaNow]:
    c1 = Counter(df.Group_Label)
    #c2 = Counter(df.Locality)
    c3 = Counter(df.Region)

    for c in [c1,c3]:
        print(c.most_common(50), len(c))
        print('-' * 50)
    
reichMeta[['Id', 'Group_Label', 'Locality', 'Region']]


[('Spain_C', 77), ('Germany_EN_LBK', 54), ('France_MN', 43), ('Spain_MLN', 42), ('Switzerland_LN', 39), ('Germany_BellBeaker', 37), ('Czech_BellBeaker', 36), ('Russia_MLBA_Sintashta', 35), ('Italy_Imperial.SG', 35), ('Spain_EBA', 34), ('Taiwan_Hanben_IA', 34), ('Estonia_EarlyViking.SG', 34), ('Israel_MLBA', 32), ('Scotland_N', 30), ('Germany_Lech_EBA', 30), ('Serbia_IronGates_Mesolithic', 29), ('Pakistan_Loebanr_IA', 29), ('Denmark_Viking.SG', 29), ('Ireland_MN.SG', 26), ('England_C_EBA', 25), ('Turkey_Alalakh_MLBA', 25), ('India_RoopkundA', 23), ('Russia_Afanasievo', 22), ('Pakistan_Katelai_IA', 22), ('England_N', 21), ('Germany_EarlyMedieval.SG', 21), ('Turkmenistan_C_Geoksyur', 20), ('Mongolia_LBA_Khovsgol_6', 20), ('Turkmenistan_Gonur_BA_1', 19), ('England_BellBeaker', 19), ('Cuba_CanimarAbajo_Archaic', 19), ('Italy_Sardinia_EBA', 18), ('England_Viking.SG', 18), ('Jordan_LBA', 17), ('England_N.SG', 17), ('Germany_Lech_EBA_lc', 17), ('Turkey_Arslantepe_LateC', 17), ('Russia_MLBA_Kra

,Id,Group_Label,Locality,Region
0,MAL-005,Malawi_Yao,Dedza // Yao,Malawi
1,MAL-009,Malawi_Yao,Machinga // Yao,Malawi
2,MAL-011,Malawi_Chewa,Mchinga // Chichewa,Malawi
3,MAL-012,Malawi_Chewa,Salima // Chichewa,Malawi
4,MAL-014,Malawi_Chewa,Nambuma // Chichewa,Malawi
...,...,...,...,...
13192,VK94.SG,Denmark_Viking.SG,"Sealand, Gl. Lejre",Denmark
13193,VK95.SG,Iceland_Viking.SG,Hofstadir,Iceland
13194,VK98.SG,Iceland_Viking.SG,Hofstadir,Iceland
13195,VK99.SG,Iceland_Viking.SG,Hofstadir,Iceland


In [17]:
reichMeta = reichMeta[['Id', 'Group_Label']].rename(columns={'Group_Label':"Region"}).set_index('Id')
reichMeta

,Region
Id,
MAL-005,Malawi_Yao
MAL-009,Malawi_Yao
MAL-011,Malawi_Chewa
MAL-012,Malawi_Chewa
MAL-014,Malawi_Chewa
...,...
VK94.SG,Denmark_Viking.SG
VK95.SG,Iceland_Viking.SG
VK98.SG,Iceland_Viking.SG


In [18]:
meta = pd.concat([yemenMeta, chadContext, pakistan, reichMeta])
meta.to_csv("Metadata/allmetaRegions.csv")

In [19]:
def maxComponent(sample, threshold={'ME':.9}, defaultThreshold=0.5):
    maxComp = sample.iloc[:8].idxmax()
    if sample[maxComp] > threshold.get(maxComp, defaultThreshold):
        return 'UAE_' + maxComp
    return 'UAE_Misc'

## selecting UAE members with high ME component, high CSA component, high Africa component
q = pd.read_csv('UAE/uae_hgdp1LDa.8.Q', header=None, sep='\s+', index_col=9)

qcol = 'SSA NA ME CSA EAS OCE EUR AMR'.split()
q.columns = qcol + ['Fam', 'P1', 'P2', 'P3', 'P4']
q.index.name = 'Id'
uaeAdmix = q[~q.index.str.startswith('HGDP')]
uaeAdmix.head()

uaeAdmix['maxComponent'] = [maxComponent(row) for i,row in uaeAdmix.iterrows()]


/bmshare/ahenschel/gemini/anaconda/envs/impute/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [20]:
coarseLevel = 'Admix' # 'POB' 'coarse'
uae = pd.read_csv('UAE/uaeMeta2.csv')

## Uniform distribution over all Emirates of strongly admixed samples?
### Chi-square ($\chi^2$) test with threshold for largest component: 50%
* $H_0$: are those samples with high ME component equally distributed (rejected, p=3.9e-6)
* Middle East%  high in Al Ain, Abu Dhabi

* $H_0$: are those samples with high CSA component equally distributed (rejected, p=2.9e-7)
* Central/South Asia% high in Dubai, Sharjah

* $H_0$: are those samples with high SSA component equally distributed (not rejected, p=0.55)
### Chi-square ($\chi^2$) test with threshold for largest component: 90%
* $H_0$: are those samples with high ME component equally distributed (rejected, p=1.98e-10)


In [196]:
# reformating/joining uae table

uae2 = uae.copy()
uae2 = uae2.set_index('sample ID ')
uae2.index.name = "Id"
uae2.index = uae2.index.astype(str)
uae2 = pd.concat([uaeAdmix, uae2], axis=1)



uae2.head()


,SSA,NA,ME,CSA,EAS,OCE,EUR,AMR,Fam,P1,P2,P3,P4,maxComponent,Unnamed: 0,Place of birth,From,pob
12551,0.152119,0.019683,0.487265,0.336686,0.003883,0.000344,0.000010,0.000010,1.0,0.0,0.0,0.0,-9.0,UAE_Misc,886.0,Oman,UAE,oman
120001,0.000010,0.000010,0.660677,0.337248,0.000010,0.000010,0.000010,0.002024,1.0,0.0,0.0,0.0,-9.0,UAE_Misc,566.0,Dubai,Dubai,dubai
12572,0.001121,0.078611,0.530975,0.120977,0.000010,0.000238,0.268058,0.000010,2.0,0.0,0.0,0.0,-9.0,UAE_Misc,1025.0,Palestine,UAE,palestine
120002,0.118916,0.027370,0.222200,0.611607,0.000010,0.000010,0.019877,0.000010,2.0,0.0,0.0,0.0,-9.0,UAE_CSA,802.0,Sharjah,Sharjah,sharjah
12586,0.022887,0.000010,0.971673,0.000010,0.000010,0.005390,0.000010,0.000010,3.0,0.0,0.0,0.0,-9.0,UAE_ME,225.0,Abu Dhabi,Abu Dhabi,abu_dhabi


In [197]:
## needs fix: recalc uaeME...

df = pd.DataFrame([Counter(uae2.pob),
                   Counter(uae2.loc[uae2.maxComponent=='UAE_ME'].pob), 
                   Counter(uae2.loc[uae2.maxComponent=='UAE_CSA'].pob), 
                   Counter(uae2.loc[uae2.maxComponent=='UAE_SSA'].pob)]).T.fillna(0)

df.columns='Total ME CSA SSA'.split()
df = df.loc['ajman dubai sharjah rak abu_dhabi al_fujairah al_ain'.split()]

totals = df.sum()

df["ME_exp"] = (df.Total*totals.ME)/totals.Total
df["MEpct"] = (df.ME/df.Total)

df["CSA_exp"] = (df.Total*totals.CSA)/totals.Total
df["CSApct"] = (df.CSA/df.Total)

df["SSA_exp"] = (df.Total*totals.SSA)/totals.Total
df["SSApct"] = (df.SSA/df.Total)

df = df[sorted(df.columns.values)]
df.sort_values(by='MEpct', ascending=False)


#df.index

,CSA,CSA_exp,CSApct,ME,ME_exp,MEpct,SSA,SSA_exp,SSApct,Total
al_ain,6.0,16.538089,0.061856,37.0,17.476421,0.381443,4.0,2.111245,0.041237,97.0
abu_dhabi,38.0,60.185006,0.107649,86.0,63.599758,0.243626,5.0,7.683192,0.014164,353.0
ajman,3.0,1.534462,0.333333,1.0,1.621524,0.111111,0.0,0.195889,0.000000,9.0
dubai,42.0,22.164450,0.323077,11.0,23.422007,0.084615,4.0,2.829504,0.030769,130.0
rak,25.0,19.436518,0.219298,8.0,20.539299,0.070175,1.0,2.481258,0.008772,114.0
sharjah,23.0,14.833132,0.264368,6.0,15.674728,0.068966,3.0,1.893591,0.034483,87.0
al_fujairah,4.0,6.308343,0.108108,0.0,6.666264,0.000000,1.0,0.805320,0.027027,37.0


In [198]:
chisquare(df.ME, df.ME_exp)

Power_divergenceResult(statistic=56.81932181696892, pvalue=1.9874348434453936e-10)

In [199]:
chisquare(df.CSA, df.CSA_exp)

Power_divergenceResult(statistic=40.97733049155432, pvalue=2.925594459032629e-07)

In [200]:
chisquare(df.SSA, df.SSA_exp)

Power_divergenceResult(statistic=4.884658503998933, pvalue=0.5586895300407138)

In [201]:
totals

Total    827.0
ME       149.0
CSA      141.0
SSA       18.0
dtype: float64

In [21]:
coarseLevel = 'pob'
if coarseLevel == 'coarse':
    uae = uae[['sample ID ']]
    uae['Region'] = 'UAE'
elif coarseLevel == 'Admix':
    uae = uaeAdmix.reset_index(level=0)[['Id', 'maxComponent']]
else:
    uae = uae[['sample ID ', 'pob']]

    
uae.columns = ['Id', 'Region']    
uae = uae.set_index('Id')
uae.index = uae.index.map(str)
uae

,Region
Id,
12866,al_fujairah
12951,al_fujairah
13085,al_fujairah
10730,al_fujairah
10218,al_fujairah
...,...
10865,um_alquwain
120016,um_alquwain
120042,um_alquwain


In [203]:
uae

,Region
Id,
12551,UAE_Misc
120001,UAE_Misc
12572,UAE_Misc
120002,UAE_CSA
12586,UAE_ME
...,...
120421,UAE_Misc
10411,UAE_Misc
10449,UAE_ME


In [22]:
allRegions = pd.concat([yemenMeta, chadContext, pakistan, reichMeta, uae]).dropna()
allRegionsNoUAE = pd.concat([yemenMeta, chadContext, pakistan, reichMeta]).dropna()

In [25]:
allRegions.to_csv('Metadata/allRegions.csv')

In [211]:
opops = [pop for pop, count in Counter(allRegionsNoUAE.Region).most_common() if (not pop.startswith('Ignore') and count>=10)]

In [214]:
## What is the closest population to UAE_ME/UAE_CSA/Hdr/Rsa/Ibb/Tiz/Sad

parfileText = """genotypename:   swAsia_uae_reichHO_final_QC.geno
snpname:        swAsia_uae_reichHO_final_QC.snp
indivname:      swAsia_uae_reichHO_final_QC4.ind
popfilename:    %s
inbreed:	YES"""

uaepops = 'UAE_ME/UAE_CSA/Hdr/Rsa/Ibb/Tiz/Sad'.split('/')
for pop in uaepops:
    popfilename = f'popfile_{pop}_vAll.txt'
    with open(popfilename, 'w') as popfile:
        for pop2 in opops:
            print(pop, pop2, "Mbuti", file=popfile)
    with open(f'parfile_qp3Pop_{pop}', 'w') as parfile:
        print(parfileText % popfilename, file=parfile)
    cmd = f"qp3Pop -p parfile_qp3Pop_{pop} > qp3Pop_{pop}.log &"
    print(cmd)


qp3Pop -p parfile_qp3Pop_UAE_ME > qp3Pop_UAE_ME.log &
qp3Pop -p parfile_qp3Pop_UAE_CSA > qp3Pop_UAE_CSA.log &
qp3Pop -p parfile_qp3Pop_Hdr > qp3Pop_Hdr.log &
qp3Pop -p parfile_qp3Pop_Rsa > qp3Pop_Rsa.log &
qp3Pop -p parfile_qp3Pop_Ibb > qp3Pop_Ibb.log &
qp3Pop -p parfile_qp3Pop_Tiz > qp3Pop_Tiz.log &
qp3Pop -p parfile_qp3Pop_Sad > qp3Pop_Sad.log &


In [207]:
allRegions

,Region
Id,
3577STDY6068360,Dal
3577STDY6068361,Ibb
3577STDY6068362,Rsa
3577STDY6068363,Rsa
3577STDY6068364,Rsa
...,...
120421,UAE_Misc
10411,UAE_Misc
10449,UAE_ME


In [209]:

with open('swAsia_uae_reichHO_final_QC4.ind', 'w') as ind:
    for line in open('swAsia_uae_reichHO_final_QC.ind_bak'):
        fields=line.split()
        f = fields[0]
        if f.startswith('urn:wtsi'):
            f = '_'.join(f.split('_')[2:])        
        region = allRegions['Region'].get(f, 'Control')
        print(line[:line.index(fields[2])] + region, file=ind)


In [158]:

pops = {'levantPops': 'Egyptian Druze Druze.SDG Palestinian BedouinB BedoiunA.SDG Iranian Jordanian'.split(),
        'arabPops': 'UAE_ME Yemeni_Highlands Yemeni_Desert2 Yemeni_Northwest Rsa Ibb Hdr Tiz Dhm Abyn Sad Mhw Lahj Jwf San Mrb Amr Haj Shb Somali.DG1 Eritrea Saudi'.split()
       }

B = 'Chad' # 'Dinka' 'Shaigi.WGA'
outgroup = 'Mbuti' # has few members only

for popgroup, pops in pops.items():
    with open(f'swAsia_uae_reichHO_final_QC_{popgroup[:-4]}_{B}_{outgroup}.pop', 'w') as popfile:        
        for pop in pops:
            print(pop, B, outgroup, file=popfile)
        

In [150]:
ls -lt |head

total 5413508
-rw-rw-r--. 1 ahenschel ahenschel        425 Oct 22 17:42 swAsia_uae_reichHO_final_QC_arab.pop
-rw-rw-r--. 1 ahenschel ahenschel        134 Oct 22 17:42 swAsia_uae_reichHO_final_QC_levant.pop
-rw-rw-r--. 1 ahenschel ahenschel     151279 Oct 22 17:42 fixIndivPopulation.ipynb
-rw-rw-r--. 1 ahenschel ahenschel       5599 Oct 22 16:55 jupyter3.log
-rw-rw-r--. 1 ahenschel ahenschel     611192 Oct 22 16:55 metadataExplorer.ipynb
-rw-rw-r--. 1 ahenschel ahenschel     588927 Oct 22 16:07 swAsia_uae_reichHO_final_QC3.ind
drwxrwxr-x. 1 ahenschel ahenschel        980 Oct 22 12:43 UAE/
-rw-rw-r--. 1 ahenschel ahenschel    2898205 Oct 22 11:04 zalloua480_reich_PCA.ipynb
-rw-rw-r--. 1 ahenschel ahenschel          0 Oct 22 10:51 jupyter4.log
ls: write error


In [151]:
!more swAsia_uae_reichHO_final_QC_levant.pop

Druze Chad Yoruba
Palestinian Chad Yoruba
BedouinB.SDG Chad Yoruba
BedoiunA.SDG Chad Yoruba
Iranian Chad Yoruba
Jordanian Chad Yoruba


In [59]:
%pprint
len(sorted(chadContext.index))

Pretty printing has been turned ON


276

In [54]:
chadContext.index

,Region
Id,
yemcha6089628,Chad
yemcha6089629,Chad
yemcha6089630,Chad
yemcha6089631,Chad
yemcha6089632,Chad
...,...
yemcha6089909,Chad
yemcha6089910,Chad
yemcha6089912,Chad


In [4]:
from collections import Counter
Counter('_'.join(line.split()[2:]) for line in open('swAsia_uae_reichHO_final_QC.ind')).most_common()


[('Russia', 1705),
 ('China', 1138),
 ('UAE', 996),
 ('Spain', 680),
 ('Pakistan', 657),
 ('Italy', 538),
 ('USA', 446),
 ('Great_Britain', 412),
 ('Israel', 369),
 ('Germany', 356),
 ('Nigeria', 352),
 ('France', 286),
 ('India', 254),
 ('Chad', 245),
 ('Turkey', 241),
 ('Kenya', 220),
 ('Hungary', 215),
 ('Sweden', 193),
 ('Japan', 176),
 ('Peru', 170),
 ('Kazakhstan', 166),
 ('Vietnam', 148),
 ('Mongolia', 139),
 ('Puerto_Rico', 130),
 ('Iran', 128),
 ('Dominican_Republic', 127),
 ('Mexico', 123),
 ('Finland', 122),
 ('Gambia', 121),
 ('Uzbekistan', 113),
 ('Poland', 110),
 ('Colombia', 108),
 ('Yemen', 107),
 ('Sri_Lanka', 103),
 ('Ukraine', 98),
 ('Estonia', 96),
 ('Bangladesh', 96),
 ('Barbados', 96),
 ('Sierra_Leone', 95),
 ('Denmark', 95),
 ('Turkmenistan', 92),
 ('Czech_Republic', 87),
 ('Switzerland', 80),
 ('Papua_New_Guinea', 76),
 ('Brazil', 72),
 ('Greece', 69),
 ('Taiwan', 69),
 ('Algeria', 66),
 ('Nepal', 66),
 ('Lebanon', 65),
 ('Ireland', 63),
 ('Bulgaria', 58),
 ('Ic

In [57]:
df=pd.DataFrame({'Exp':[25,25,25,25], 'Obs':[20,25,20,35]})

In [56]:
import scipy

In [58]:
from scipy.stats import chisquare
chisquare(df.Obs, df.Exp)

Power_divergenceResult(statistic=6.0, pvalue=0.11161022509471268)